In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model


In [18]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


In [19]:
model=load_model('rnn_model_imdb.h5')
model.summary()
model.get_weights()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

[array([[ 0.00489829, -0.0403402 , -0.03745616, ..., -0.02061587,
          0.00336962,  0.0154821 ],
        [ 0.00824389, -0.01173686, -0.03647309, ...,  0.03443617,
         -0.00274033,  0.07754651],
        [-0.00663608,  0.00076077, -0.0238536 , ..., -0.02381147,
         -0.05540113,  0.03894865],
        ...,
        [ 0.02828744, -0.04808999, -0.04842664, ...,  0.04342968,
          0.01361823, -0.0191479 ],
        [ 0.06823451,  0.00955141,  0.00985605, ...,  0.02466028,
          0.01279059, -0.00706585],
        [ 0.05994377, -0.06142515, -0.02568587, ..., -0.01253543,
         -0.07276994,  0.03337575]], dtype=float32),
 array([[ 0.18514714,  0.04097956,  0.01057247, ...,  0.13851778,
          0.24978688, -0.00423423],
        [ 0.1563574 , -0.01193191, -0.14224845, ..., -0.12947015,
          0.09668087,  0.07293073],
        [-0.15543269, -0.14889222, -0.07354352, ...,  0.01095848,
         -0.09644442, -0.12604408],
        ...,
        [ 0.0841879 , -0.04431619,  0.2

In [20]:
# Functions to decode the review and to pad the review
import re

def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

def preprocess_text(text):
    # Clean the input text
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text.lower())
    words = text.split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review


In [21]:
# Prediction function

def predict_sentiment(text):
    padded_review = preprocess_text(text)
    pred = model.predict(padded_review)
    sentiment = 'Positive' if pred[0][0] > 0.5 else 'Negative'
    return sentiment,pred[0][0]

In [24]:
# Test with sample
text = "The movie was utter trash, I would not recommend it to anyone."
sentiment,score = predict_sentiment(text)

print(f'Review: {text}')
print(f'Sentiment: {sentiment}')
print(f'Confidence: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Review: The movie was utter trash, I would not recommend it to anyone.
Sentiment: Negative
Confidence: 0.1140713319182396


In [25]:
# Test with sample
text = "The movie was a masterpiece, I would recommend it to everyone."
sentiment,score = predict_sentiment(text)

print(f'Review: {text}')
print(f'Sentiment: {sentiment}')
print(f'Confidence: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Review: The movie was a masterpiece, I would recommend it to everyone.
Sentiment: Positive
Confidence: 0.9559293985366821
